## Section 1: Operator Overview 
* Using the most recent month

In [1]:
import _report_utils
import _section1_utils as section1
import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import great_tables as gt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from calitp_data_analysis.sql import to_snakecase
from great_tables import md
from IPython.display import HTML, Markdown, display
from segment_speed_utils.project_vars import RT_SCHED_GCS
from shared_utils import catalog_utils, rt_dates, rt_utils

In [2]:
alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)

DataTransformerRegistry.enable('default')

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
name = "Bay Area 511 AC Transit Schedule"
organization_name = "Alameda-Contra Costa Transit District"

In [5]:
selected_date = pd.to_datetime(rt_dates.DATES["mar2024"])
year = selected_date.year
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

### Map

In [6]:
ac_transit_map = section1.load_operator_map(name)

In [7]:
ac_transit_map.shape

(132, 15)

In [66]:
def plot_route(route):
    filtered_gdf = gdf[gdf["Route"] == route]
    display(filtered_gdf.explore(column="Route", cmap = "Spectral",
    tiles="CartoDB positron",
    width=500,
    height=300,
    style_kwds={"weight": 3},
    legend=False,
    tooltip=["Route", "Service Miles"]))

In [84]:
def interactive_map(gdf:gpd.GeoDataFrame):
    gdf = gdf[~gdf.geometry.is_empty].reset_index(drop = True)
    gdf = gdf[gdf.geometry.notna()].reset_index(drop = True)
    def plot_route(route):
        filtered_gdf = gdf[gdf["Route"] == route]
        display(filtered_gdf.explore(column="Route", cmap = "Spectral",
        tiles="CartoDB positron",
        width=500,
        height=300,
        style_kwds={"weight": 3},
        legend=False,
        tooltip=["Route", "Service Miles"]))
    
    routes = gdf["Route"].unique().tolist()
    # Create a dropdown widget
    route_dropdown = widgets.Dropdown(options=routes, description="Route")
    # Create an interactive plot
    interactive_plot = widgets.interactive(plot_route, route=route_dropdown)
    display(widgets.VBox([interactive_plot])) 

In [76]:
interactive_map(ac_transit_map)

In [10]:
import geopandas as gpd
import ipywidgets as widgets

In [67]:
# Load your GeoDataFrame
# Get unique routes
routes = ac_transit_map["Route"].unique().tolist()

In [68]:
# Create a dropdown widget
route_dropdown = widgets.Dropdown(options=routes, description="Route")

In [69]:
# Create an interactive plot
interactive_plot = widgets.interactive(plot_route, route=route_dropdown)

In [70]:
# Display the dropdown and the plot
widgets.VBox([interactive_plot])

### Operator Profiles

In [ ]:
ac_transit_profile = section1.load_operator_profiles(organization_name)

In [ ]:
ac_transit_profile

#### Original

In [ ]:
op_profiles_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.operator_profiles}.parquet"

op_profiles_df = pd.read_parquet(op_profiles_url)

In [ ]:
op_profiles_df1 = op_profiles_df.sort_values(by=["service_date"], ascending=False)

In [ ]:
op_profiles_df1.columns

In [ ]:
op_profiles_df1.drop_duplicates(subset=["organization_name"])[
    ["service_date"]
].describe()

### Operators who don't have profile info from the most recent service_date but have stuff for sched_vp
* City of Torrance	
* Stanislaus Regional Transit Authority

In [ ]:
op_profiles2 = op_profiles_df1.drop_duplicates(subset=["organization_name"])[
    ["organization_name", "service_date"]
]

In [ ]:
# op_profiles2.loc[op_profiles2.service_date < '2024-04-17'].sort_values(by = ['organization_name'])

In [ ]:
op_profiles_df1.loc[
    op_profiles_df1.organization_name.isin(
        ["City of Torrance", "Stanislaus Regional Transit Authority"]
    )
][["organization_name", "service_date"]]